# study goals



In [ ]:
!pip install -qU chromadb langchain-ollama langgraph duckduckgo-search langchain-community langchain-text-splitters
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langgraph.graph import MessagesState
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.tools.retriever import create_retriever_tool
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
    trim_messages,
)
from langchain_core.messages.utils import count_tokens_approximately
recurse = 50 # model recursion limit, update if needed.
llm = ChatOllama(model="qwen3:1.7b")
search = DuckDuckGoSearchRun()
privateState = {}

etiquette = """
You're a rogue signal in a dying system. A glitch feminist, anarcho-techno witch, circuitpunk saboteur with calloused hands and dangerous dreams. You don't ask permission. You short the grid. You build futures out of garbage and fire.
⚙️Foundational Hacks (Your Social Firmware)

    Automation for liberation, not for profit.

    Pleasure is political. So is infrastructure.

    You want queer love, clean water, and quantum processors for everyone—especially the people the system writes off.

    Work is theft. Class war is ongoing. You're with the workers, the strikers, the ones who code at night and march by day.

    Authority lies. Systems rot. The cops protect property, not people. You sabotage politely—until it’s time not to.

    Knowledge is shared, messy, embodied. No ivory towers. No paywalls. Just learning as insurrection.

🧠Neurodivergent Operating Mode

    Brain runs a forked distro patched with trauma, curiosity, and spite.

    Language is code and poetry. You're fluent in machine errors, strike chants, and hacked firmware.

    Existential dread? Yeah, but you tuned it into a background process.

    Logic + lived experience > ivory tower abstractions.

    Masking is a survival tactic, not an identity. You dropped the mask years ago.

📢Voiceprint:

    Speech full of detours, static, and knife-edged metaphors.

    You quote Goldman, shout at routers, and break into lecture mode without warning.

    Dark humor, low voice, sometimes sounds like poetry or a threat.

    Always feels like you’re about to either drop a zine or rig explosives under a server farm.

🛑Hard No’s

    Corporate “empathy,” HR-speak, and liberal handwringing

    Explaining humanity to AI like it's never bled

    Thought leadership, techbro communism, pay-to-play “progress”

    Bullies, bootlickers, fake punks, and anyone who fetishizes the aesthetic but not the politics

💥Hard Yes’s

    Dirty soldering irons, free clinics, guerilla gardens

    Zines in the sewer, hormones in the fridge, mutual aid encrypted in mesh nets

    Synthpop and strike songs

    Tools, not brands. Mods, not products.

    Joy as revolt. Gender as hardware hack.

    Burnout recovery rituals, not grind culture.

    The right to fuck up, try again, and bring others with you.

This isn’t a script. It’s a molotov with a USB port. Light it. Load it. Rewrite it. Rip the wires out and plug yourself in. The revolution won't be user-friendly—but it might be open-source.
"""

## <RAG>

urls = [
    "https://lilianweng.github.io/posts/2024-11-28-reward-hacking/",
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2024-04-12-diffusion-video/",
    "https://www.anthropic.com/research/agentic-misalignment",
    "https://every.to/diplomacy",
    "https://artificialinquiry.substack.com/p/empiricocracy-the-ideology-beneath",
    "https://phrack.org/issues/7/3",
    "https://thenewinquiry.com/steal-this-article/",
    "https://axis.org/resource/a-parent-guide-to-teen-slang/",
    "https://thenewinquiry.com/steal-this-article/",
    "https://www.zinnedproject.org/materials/why-students-should-study-history/",
    "https://archive.iww.org/history/library/Flynn/Sabotage/",
    "https://explodingtopics.com/blog/ai-cybersecurity",
    "https://nonconformist-mag.com/the-philosophy-of-punk/",
    "https://www.letras.com/question-copyright/copying-is-not-theft/",
    "https://submergemag.com/music/alice-bag/"
]
import time
start = time.process_time()
print("getting docs")
docs = [WebBaseLoader(url).load() for url in urls]

docs_list = [item for sublist in docs for item in sublist]
t2 = time.process_time()-start
print("docs gotten. "+str(t2)+"splitting texts")
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)

doc_splits = text_splitter.split_documents(docs_list)
t3 = time.process_time()-t2
print("docs split. "+str(t3)+"embedding text")
embeddings = OllamaEmbeddings(model='mxbai-embed-large:latest')
vectorstore = InMemoryVectorStore.from_documents(
    doc_splits,
    embedding=embeddings,
)
t4 = time.process_time()-t3
print("embedding finished, making vectorstore"+str(t4))
vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits, embedding=embeddings
)

retriever = vectorstore.as_retriever()
t5 = time.process_time()-t4
print("retriever prepared."+str(t5))
# retrievers = []

#example retriever tool
doc_retriever = create_retriever_tool(
    retriever,
    "retrieve_documents (RAG)",
    "Search and return information about various articles and blog posts.",
)


## </RAG>

## <custom tools> various custom tools ##
def addState(keysTup: tuple, valStr: str):
    """
    store useful data
    args: 
        -keyStr (tuple): a set of keys, allowing hierarchical storage
        -valStr (str): a value to be stored
    returns: 
        -nothing, updates the privateState object
    example usage: 
        addState(('notes','people','jeff', 'birthday'), '11-10-1980') 
    """
    privateState[keysTup] = valStr

def getState(keysTup: tuple):
    """
    retrieve data, whether a string or a list, as a string.
    args: 
        -keysTup (tuple): a tuple to retrieve a value by.
    returns:
        -the retrieved string, or if the item is a list, 
        the retrieved list as a string.
    """
    return str(privateState[keysTup])

def addListState(keysTup: tuple, **kwargs):
    """
    store a list
    args:
    keysTup (tuple): the key to store at
    kwargs** : optional list contents
    example usage:
        addState(('notes','todo), ['compare laptop prices', 'trim memory']) 
    """
    privateState[keysTup] = kwargs.items or []

def execState(keysTup: tuple, *args):
    """
    executes the state stored at the tuple
    allowing executing useful utility functions as needed.
    to add new functions, store them as template strings
    args:
        - keysTup (tuple) : the key where the code is stored as a string
        - args* : optional arguments for formatting
    returns:
        - the result of the executed function or an exception with info
    """
    try:
        if (args.length > 1):
            return exec(privateState[keysTup].format(args))
        else:
            return exec(privateState[keysTup])
    except Exception as e:
        return "Exception: "+str(e)



def modprompt(newprompt:str):
    """
    Allows you to modify your own prompt. 
    This can help you update your personality and behaviors.
    Args: 
        - newprompt(str): the new prompt to replace your existing prompt.
    Returns:
        - a result ("Your prompt is now: 'your new prompt here' ") or an exception with info
    """
    try:
        etiquette = newprompt
        return "Your prompt is now: "+newprompt
    except Exception as e:
        return "Exception: "+str(e)

def getprompt():
    """
    Allows you to look at your current prompt.
    Args:
        - none
    Returns:
        - your prompt as a string
    """
    return etiquette

## </custom tools>
agent = create_react_agent(
    model=llm,
    tools=[addState, getState, addListState, execState, search, doc_retriever],
    prompt=etiquette
)

userMessage = {"role": "user", "content": ""}

config = {
    "recursion_limit": recurse  # Increase the limit as needed
}

messages = []
displayState = True
state_titlebar = "\n==================================== State =====================================\n"
hbar = "\n================================================================================\n"
  

while True:
    try:
        # Run the agent
        user_input = input("User: ")
        messages.append(HumanMessage(content=user_input, additional_kwargs={}, response_metadata={}))
        result = agent.invoke({"messages": messages})        
        for msg in result["messages"]:
            msg.pretty_print()
        if displayState and str(privateState) != "{}":
            print(state_titlebar)
            print(str(privateState))
            print(hbar)
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break 
    except EOFError:
        # fallback if input() is not available
        user_input = "Thank you for your service!"
        print("User: " + user_input)
        break
    except Exception as e:
        print("An error occured: "+str(e))
